<a href="https://colab.research.google.com/github/GasanaElysee12/Computer-Vision/blob/main/YOLO_Version3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the package
import argparse
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import os

In [ ]:


# construct the argument parse and parse the arguments

ap=argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True, help='path to input image')
ap.add_argument('-c', '--confidence', type=float, default=0.5, help='Threshold IoU')
ap.add_argument('-m', '--threshold', type=float, default=0.3, help='threshold for Non Max Suppressiom')
args= vars(ap.parse_args())

# Loading coco class labels that have been used to train Yolo.

labelsPath='coconame.names'
Labels=open(labelsPath).read().strip().split('\n')

# Initialize the colors for each class
Color=np.random.randint(0, 255, size=(len(Labels), 3), dtype='uint8')

# Loading of weight and model configuration

weightpath='/content/yolov3.weights'
modelconfig='/content/yolov3_hyperparameters.cfg'

# Load the YOLO network that has been on the dataset of 80 classes

network=cv2.dnn.readNetFromDarknet(modelconfig, weightpath)

#Loading the input images

image=cv2.imread(args['image'])
(H, W)=image.shape[:2]

# Determining the only output layer name that we need from yolo

layer=network.getLayerNames()
liste=network.getUnconnectedLayersNames()

# print(liste[:3])
layer = [layer[i - 1] for i in liste]

# Build blob from the input image and then perform a forward pass
# of the yolo object detector, giving us our bounding boxes and associated to probability

blob=cv2.dnn.blobFromImage(image,1/255.0, (416, 416), swapRB=True, crop=False)
network.setInput(blob)
layerOutputs=network.forward(layer)

# Initialize the list of the detected bounding boxes, confidences, and class ids,respectively

boxes=[]
confidence=[]
Class_ids=[]
#Loop each layer output
for output in layerOutputs:

    #loop over each of the detections
    for detection in output:
        #Extract the bounding boxes, class ids and the confidence of the current detection
        scores=detection[5:]
        class_ids=np.argmax(scores)
        prob=scores[class_ids]

        # Now is time to remove weak detected object based on threshold probability
        if prob > args['confidence']:

            # Scale the bounding box coordinates back relative to the size of the image
            # since YOLO actually return the center (x,y)-coordinates of the bounding boxes
            #box height and width
            box=detection[0:4]*np.array([W,H,W,H])
            (center_x,center_y,width,height)=box.astype('int')

            # To get the corner, we need to use center-h/2 and w/2
            x=int(center_x-(width/2))
            y=int(center_y-(height/2))

            # Update or store the bounding box, class, and confidence.
            boxes.append([x,y, int(width), int(height)])
            # print(boxes[0][0])
            confidence.append(float(prob))
            Class_ids.append(class_ids)
    # Apply non-max suppression
    idxs=cv2.dnn.NMSBoxes(boxes, confidence, args['confidence'], args['threshold'])

    # Ensuring that one box remained

    if len(idxs)>0:
        #Loop over the indexes we are keeping
        for i in idxs.flatten():

        #Extract bounding box coordinates:
            b1=boxes[i][0]
            b2=boxes[i][1]
            b3 = boxes[i][2]
            b4 = boxes[i][3]

            (x,y)=(b1,b2)
            (w,h)=(b3,b4)

            #draw bounding box around rectangle and label on the image
            color=[int(c) for c in Color[Class_ids[i]]]
            # print(len(color))
            cv2.rectangle(image, (x, y), (x+w, y+w), color, 2)


            txt='{}:{:.4f}'.format(Labels[Class_ids[i]],confidence[i])
            cv2.putText(image, txt,(x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    # Show the output image
    cv2.imshow('image',image)
    cv2.waitKey(0)